<a href="https://colab.research.google.com/github/Lavanya8484/Celebal_Summer_Internship/blob/main/Week-8/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q pandas numpy faiss-cpu sentence-transformers transformers gradio


In [5]:
from google.colab import files
uploaded = files.upload()  # Upload the "Training Dataset.csv"


Saving loan_data_set.csv to loan_data_set (1).csv


In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

csv_file = "Training Dataset.csv"  # Ensure this matches the uploaded name
df = pd.read_csv(csv_file)

# Convert each row into a descriptive string
texts = []
for i, row in df.iterrows():
    try:
        info = f"Gender={row['Gender']}, Married={row['Married']}, Education={row['Education']}, "
        info += f"Self_Employed={row['Self_Employed']}, Credit_History={row['Credit_History']}, "
        info += f"LoanAmount={row['LoanAmount']}, Outcome={row['Loan_Status']}"
        texts.append(info)
    except:
        continue

# Embed with SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, show_progress_bar=True)

# Create FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

# Save data for retrieval
text_data = texts.copy()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
# Load retrieval model again if not already loaded
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the FLAN-T5 model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
generation_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Retrieval function
def retrieve(query, k=3):
    q_embed = retrieval_model.encode([query])
    D, I = index.search(np.array(q_embed), k)
    return [text_data[i] for i in I[0]]

# Generation function
def generate_answer(query):
    context = " ".join(retrieve(query))
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = generation_model.generate(input_ids, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [19]:
query = "Will a person with a credit history of 1 get loan approval?"
print("Q:", query)
print("A:", generate_answer(query))


Q: Will a person with a credit history of 1 get loan approval?
A: No


In [15]:
import gradio as gr

def chat_interface(query):
    return generate_answer(query)

gr.Interface(fn=chat_interface, inputs="text", outputs="text", title="Loan Approval RAG Q&A Bot").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81af5c5c5d5b94e036.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
